In [ ]:
# %matplotlib ipympl

import os
import glob
import torch
from torch import nn
import networkx as nx
from torch.utils.data import DataLoader, ConcatDataset, random_split
import torch.optim as optim
import tqdm
import time
import csv
from learning.model.stateflow import DWStateFlowModel
from learning.data.stateflow_dataset import DWStateFlowDataset
from learning.data.stateflow_utils import collate_stateflow_data
from learning.data.av_utils import move_to_device
from learning.data.stateflow_utils import (
    extract_input, 
    extract_winners_output_target, 
    extract_action_output_target
)

In [2]:
history_folder = "server/history/"
history_files = sorted(glob.glob(os.path.join(history_folder, "history_*.json")))

datasets = [DWStateFlowDataset(f) for f in history_files]
concatenated_dataset = ConcatDataset(datasets)

In [3]:
print(len(concatenated_dataset))

183374


In [4]:
import random
seed = random.randint(1, 10000000)
print(f"Using seed: {seed}")

generator = torch.Generator().manual_seed(seed)

train_ds, val_ds, test_ds = random_split(
    concatenated_dataset,
    [0.8, 0.1, 0.1],
)

train_loader = DataLoader(train_ds, 20000, pin_memory=True, shuffle=True, 
                         collate_fn=collate_stateflow_data, generator=generator)
val_loader = DataLoader(val_ds, 20000, pin_memory=True, shuffle=True, 
                       collate_fn=collate_stateflow_data, generator=generator)

print("train_ds", len(train_ds))
print("val_ds", len(val_ds))
print("test_ds", len(test_ds))

Using seed: 6821409
train_ds 146700
val_ds 18337
test_ds 18337


In [5]:
model_state = None
scheduler_state = None
combined_criterion_state = None
latest_epoch = -1

models_dir = "learning/stateflow_checkpoints" 
model_name = "stateflow"

# Find the latest checkpoint file (with highest epoch number)
checkpoint_files = glob.glob(os.path.join(models_dir, f"{model_name}_*.pt"))
if checkpoint_files:
    # Extract epoch numbers from filenames
    epoch_nums = [int(f.split("_")[-1].split(".")[0]) for f in checkpoint_files]
    latest_epoch = max(epoch_nums)
    latest_checkpoint = os.path.join(models_dir, f"{model_name}_{latest_epoch}.pt")
    print(f"Loading latest checkpoint: {latest_checkpoint} (epoch {latest_epoch})")
    checkpoint = torch.load(latest_checkpoint)
    model_state = checkpoint["model_state"]
    combined_criterion_state = checkpoint["combined_criterion_state"]
    scheduler_state = checkpoint["scheduler_state"]
else:
    print("No valid checkpoint files found")

start_epoch = latest_epoch + 1

Loading latest checkpoint: learning/stateflow_checkpoints/stateflow_260.pt (epoch 260)


In [7]:
stateflow_model = DWStateFlowModel()
print(stateflow_model)
num_params = sum(p.numel() for p in stateflow_model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {num_params:,}")

DWStateFlowModel(
  (norm_fn): DWNorm()
  (gat_layers): ModuleList(
    (0): GATLayer(
      (message_fn): GATMessage(
        (input_dropout): Dropout(p=0, inplace=False)
        (transform_fn): Linear(in_features=8, out_features=8, bias=False)
      )
      (attention_coef_fn): GATAttentionCoefficients(
        (transform_fn): Linear(in_features=16, out_features=1, bias=False)
        (activation_fn): LeakyReLU(negative_slope=0.01)
      )
      (attention_dropout): Dropout(p=0.35, inplace=False)
      (activation_fn): ReLU()
      (residual_proj): Linear(in_features=8, out_features=8, bias=False)
      (residual_dropout): Dropout(p=0.35, inplace=False)
    )
    (1-2): 2 x GATLayer(
      (message_fn): GATMessage(
        (input_dropout): Dropout(p=0.35, inplace=False)
        (transform_fn): Linear(in_features=8, out_features=8, bias=False)
      )
      (attention_coef_fn): GATAttentionCoefficients(
        (transform_fn): Linear(in_features=16, out_features=1, bias=False)
       

In [8]:
class StateflowLoss(nn.Module):
    # homoscedastic uncertainty trick implementation
    def __init__(self): 
        super().__init__()
        self.winner_err_log_scale = nn.Parameter(torch.tensor(0, dtype=torch.float32))
        self.action_err_log_scale = nn.Parameter(torch.tensor(0, dtype=torch.float32))
    
    def forward(self, winner_change_err, action_logit_error):
        winner_err_rescaled = \
            winner_change_err * torch.exp(-self.winner_err_log_scale) + self.winner_err_log_scale
        action_err_rescaled = \
            action_logit_error * torch.exp(-self.action_err_log_scale) + self.action_err_log_scale
        
        return winner_err_rescaled + action_err_rescaled
    

class MSELossWithOverflow:
    # add an error for output whose absolute value is greater than abs_max
    def __init__(self, abs_max, overflow_scale = 0.15):
        self.abs_max = abs_max
        self.mse_crit = nn.MSELoss()
        self.overflow_crit = nn.MSELoss()
        self.overflow_scale = overflow_scale

    def __call__(self, output, target):
        loss = self.mse_crit(output, target)
        overflow = torch.relu(torch.abs(output) - self.abs_max)
        overflow_loss = torch.mean(overflow ** 2)  # MSE
        return loss + self.overflow_scale * overflow_loss
    

# end_turn_flags = torch.tensor([d["attack_edge"][0] == -1 for d in concatenated_dataset])
# p_end_turn = torch.mean(1.0 * end_turn_flags)
# p_attack = 1 - p_end_turn
# print(p_end_turn, p_attack)  # tensor(0.2190) tensor(0.7810)
p_end_turn, p_attack = 0.25, 0.75

class IterativeCrossEntropyLoss:
    def __init__(self):
        self.ce_loss = nn.CrossEntropyLoss()

    def get_weights(self, o_logits):     
        n_actions = o_logits.shape[-1]       
        weights = torch.ones(n_actions, device=o_logits.device, dtype=o_logits.dtype)
        weights[-1] = 1 / p_end_turn  # end turn action
        p_single_edge = (1 - p_end_turn) / n_actions
        weights[:-1] = 1 / p_single_edge  # attack along edge action  
        weights = n_actions * (weights / weights.sum())  # maintain unweighted pytorch loss scale
        return weights

    def __call__(self, output, target):
        losses = []
        for o_logits, t_classes in zip(output, target):
            weights = self.get_weights(o_logits)
            loss = nn.functional.cross_entropy(o_logits, t_classes, weight=weights)
            losses.append(loss)
        return torch.mean(torch.stack(losses))


In [9]:
# Load the model before everything else so that I can freeze some layers
if model_state is not None:
    stateflow_model.load_state_dict(model_state)

stateflow_model.gat_layers[0].requires_grad_(False)
stateflow_model.gat_layers[1].reset_parameters()
stateflow_model.gat_layers[2].reset_parameters()

In [ ]:
# ── CSV set‑up: log one row per epoch ───────────────────────────────────────
csv_path = "stateflow_epoch_metrics.csv"
epoch_log_file_exists = os.path.isfile(csv_path)
if epoch_log_file_exists and os.path.getsize(csv_path) == 0:
    os.remove(csv_path)
    epoch_log_file_exists = False

csv_file = open(csv_path, "a", newline="")  # Open in append mode
csv_writer = csv.writer(csv_file)

csv_header = [
    "epoch",              # 0‑based epoch index
    "train_winner_loss",   # average winner prediction training loss for the epoch
    "train_action_loss",  # average action prediction training loss for the epoch
    "train_combined_loss", # average combined training loss for the epoch
    "val_winner_loss",     # average winner validation loss for the epoch
    "val_action_loss",    # average action validation loss for the epoch
    "val_combined_loss",  # average combined validation loss for the epoch
    "train_time_sec",     # seconds spent in training phase
    "val_time_sec",       # seconds spent in validation phase
    "total_time_sec",     # train + val
    "winner_err_log_scale", # winner error log scale parameter
    "action_err_log_scale" # action error log scale parameter
]

# ── model / optimizer prep ─────────────────────────────────────────────────

n_epochs   = 1000
device     = torch.device("cuda", 0)
time_limit = 10 * 60
winner_criterion = nn.CrossEntropyLoss()
action_criterion = IterativeCrossEntropyLoss()
combined_criterion = StateflowLoss()

if combined_criterion_state is not None:
    combined_criterion.load_state_dict(combined_criterion_state)

# Update optimizer to include both model and loss function parameters
model_optimizer = torch.optim.Adam([p for p in stateflow_model.parameters() if p.requires_grad], lr=9e-2)
combined_criterion_optimizer = torch.optim.SGD(combined_criterion.parameters(), lr=1e-2)

scheduler = torch.optim.lr_scheduler.StepLR(model_optimizer, step_size=20, gamma=0.9)

reset_scheduler = False
if not reset_scheduler and scheduler_state is not None:
    scheduler.load_state_dict(scheduler_state)
else:
    # If no scheduler state in checkpoint, step it to match the epoch
    print(f"No scheduler state found in checkpoint, stepping scheduler to epoch {start_epoch}")
    for _ in range(start_epoch):
        scheduler.step()

stateflow_model = stateflow_model.to(device)

In [11]:
def process_batch(stateflow_model, batch):
    batch = move_to_device(batch, device)
    model_out = stateflow_model(*extract_input(batch))

    winner_output, winner_target = \
        extract_winners_output_target(batch, model_out)
    action_output, action_target = \
        extract_action_output_target(batch, model_out)

    winner_loss = winner_criterion(winner_output, winner_target)
    action_loss = action_criterion(action_output, action_target)
    combined_loss = combined_criterion.forward(winner_loss, action_loss)
    return winner_loss, action_loss, combined_loss

In [12]:
# ── INITIAL (untrained) LOSS EVALUATION ───────────────────────────────────────
if not epoch_log_file_exists:
    stateflow_model.eval()
    with torch.no_grad():
        t_start = time.time()

        # avg loss on training set *without* gradient tracking
        init_train_winner_sum = torch.tensor(0.0, device=device)
        init_train_action_sum = torch.tensor(0.0, device=device)
        init_train_combined_sum = torch.tensor(0.0, device=device)
        init_train_batches = 0
        
        for tb in tqdm.tqdm(train_loader, desc="init-train"):
            winner_loss, action_loss, combined_loss = process_batch(stateflow_model, tb)
            
            init_train_winner_sum += winner_loss
            init_train_action_sum += action_loss
            init_train_combined_sum += combined_loss
            init_train_batches += 1
            
            if (time.time() - t_start) > time_limit: 
                print(f"Stopping initial training evaluation after >{time_limit} seconds.")
                break
                
        init_train_winner_loss = init_train_winner_sum.item() / init_train_batches
        init_train_action_loss = init_train_action_sum.item() / init_train_batches
        init_train_combined_loss = init_train_combined_sum.item() / init_train_batches
        t_train_done = time.time()

        # avg loss on validation set
        init_val_winner_sum = torch.tensor(0.0, device=device)
        init_val_action_sum = torch.tensor(0.0, device=device)
        init_val_combined_sum = torch.tensor(0.0, device=device)
        init_val_batches = 0
        
        for vb in tqdm.tqdm(val_loader, desc="init-val"):
            v_winner_loss, v_action_loss, v_combined_loss = process_batch(stateflow_model, vb)
            
            init_val_winner_sum += v_winner_loss
            init_val_action_sum += v_action_loss
            init_val_combined_sum += v_combined_loss
            init_val_batches += 1
            
            if (time.time() - t_train_done) > time_limit / 5:
                print(f"Stopping initial validation evaluation after >{time_limit / 5} seconds.")
                break
                
        init_val_winner_loss = init_val_winner_sum.item() / init_val_batches
        init_val_action_loss = init_val_action_sum.item() / init_val_batches
        init_val_combined_loss = init_val_combined_sum.item() / init_val_batches
        t_val_done = time.time()

    # times
    init_train_time = t_train_done - t_start
    init_val_time   = t_val_done - t_train_done
    init_total_time = t_val_done - t_start

    csv_writer.writerow(csv_header)
    # write initial row (epoch = None)
    csv_writer.writerow([
        None,
        init_train_winner_loss,
        init_train_action_loss,
        init_train_combined_loss,
        init_val_winner_loss,
        init_val_action_loss,
        init_val_combined_loss,
        init_train_time,
        init_val_time,
        init_total_time,
        combined_criterion.winner_err_log_scale.item(),
        combined_criterion.action_err_log_scale.item()
    ])
    csv_file.flush()
    print(
        f"Initial | Train Winner {init_train_winner_loss:.4f} | "
        f"Train Action {init_train_action_loss:.4f} | "
        f"Train Comb {init_train_combined_loss:.4f} | "
        f"Val Winner {init_val_winner_loss:.4f} | "
        f"Val Action {init_val_action_loss:.4f} | "
        f"Val Comb {init_val_combined_loss:.4f} | "
    )

In [13]:
for i_epoch in range(start_epoch, n_epochs):
    t_start = time.time()
    
    # Set model to training mode
    stateflow_model.train()
    
    # Training phase with gradient tracking and parameter updates
    train_winner_sum = torch.tensor(0.0, device=device)
    train_action_sum = torch.tensor(0.0, device=device)
    train_combined_sum = torch.tensor(0.0, device=device)
    train_batches = 0
    
    for tb in tqdm.tqdm(train_loader, desc=f"train-epoch-{i_epoch}"):
        # Zero the gradients
        model_optimizer.zero_grad()
        combined_criterion_optimizer.zero_grad()
        
        winner_loss, action_loss, combined_loss = process_batch(stateflow_model, tb)
        
        # Check for NaN or Inf values in loss
        if torch.isnan(combined_loss) or torch.isinf(combined_loss):
            raise ValueError(f"Combined loss contains NaN or Inf values")
            
        # Backward pass and optimize
        combined_loss.backward()
        model_optimizer.step()
        combined_criterion_optimizer.step()

        train_winner_sum += winner_loss.detach()
        train_action_sum += action_loss.detach()
        train_combined_sum += combined_loss.detach()
        train_batches += 1
        
        if (time.time() - t_start) > time_limit: 
            print(f"Stopping training after >{time_limit} seconds.")
            break
            
    train_winner_loss = train_winner_sum.item() / train_batches
    train_action_loss = train_action_sum.item() / train_batches
    train_combined_loss = train_combined_sum.item() / train_batches
    t_train_done = time.time()

    # Set model to evaluation mode for validation
    stateflow_model.eval()
    with torch.no_grad():
        # avg loss on validation set
        val_winner_sum = torch.tensor(0.0, device=device)
        val_action_sum = torch.tensor(0.0, device=device)
        val_combined_sum = torch.tensor(0.0, device=device)
        val_batches = 0
        
        for vb in tqdm.tqdm(val_loader, desc=f"val-epoch-{i_epoch}"):
            v_winner_loss, v_action_loss, v_combined_loss = process_batch(stateflow_model, vb)
            
            val_winner_sum += v_winner_loss
            val_action_sum += v_action_loss
            val_combined_sum += v_combined_loss
            val_batches += 1
            
            if (time.time() - t_train_done) > time_limit / 5:
                print(f"Stopping validation after >{time_limit / 5} seconds.")
                break
                
        val_winner_loss = val_winner_sum.item() / val_batches
        val_action_loss = val_action_sum.item() / val_batches
        val_combined_loss = val_combined_sum.item() / val_batches
        t_val_done = time.time()

    # Step the scheduler
    scheduler.step()
    
    checkpoint_path = os.path.join(models_dir, f"{model_name}_{i_epoch}.pt")
    torch.save(
        {
            "model_state": stateflow_model.state_dict(),
            "combined_criterion_state": combined_criterion.state_dict(),
            "scheduler_state": scheduler.state_dict(),
            "epoch": i_epoch
        },
        checkpoint_path
    )

    # times
    train_time = t_train_done - t_start
    val_time   = t_val_done - t_train_done
    total_time = t_val_done - t_start

    csv_writer.writerow([
        i_epoch,
        train_winner_loss,
        train_action_loss,
        train_combined_loss,
        val_winner_loss,
        val_action_loss,
        val_combined_loss,
        train_time,
        val_time,
        total_time,
        combined_criterion.winner_err_log_scale.item(),
        combined_criterion.action_err_log_scale.item()
    ])
    csv_file.flush()
    
    print(
        f"Epoch {i_epoch} | "
        f"Train Winner {train_winner_loss:.4f} | "
        f"Train Action {train_action_loss:.4f} | "
        f"Train Comb {train_combined_loss:.4f} | "
        f"Val Winner {val_winner_loss:.4f} | "
        f"Val Action {val_action_loss:.4f} | "
        f"Val Comb {val_combined_loss:.4f} | "
        f"LR {scheduler.get_last_lr()[0]:.4f}"
    )

val-epoch-261: 100%|██████████| 1/1 [00:35<00:00, 35.75s/it]


Epoch 261 | Train Winner 9.3602 | Train Action 26.7930 | Train Comb 22.4620 | Val Winner 1.3900 | Val Action 4.7365 | Val Comb 3.9536 | LR 0.0900


val-epoch-262: 100%|██████████| 1/1 [00:34<00:00, 34.31s/it]


Epoch 262 | Train Winner 1.1348 | Train Action 7.7746 | Train Comb 4.5918 | Val Winner 1.0699 | Val Action 3.0627 | Val Comb 3.3328 | LR 0.0900


val-epoch-263: 100%|██████████| 1/1 [00:37<00:00, 37.35s/it]


...

Epoch 416 | Train Winner 0.9748 | Train Action 1.5704 | Train Comb 2.4258 | Val Winner 0.9426 | Val Action 1.4363 | Val Comb 2.3073 | LR 0.0430


val-epoch-417: 100%|██████████| 1/1 [00:14<00:00, 14.00s/it]


Epoch 417 | Train Winner 0.9753 | Train Action 1.5682 | Train Comb 2.4250 | Val Winner 0.9345 | Val Action 1.4321 | Val Comb 2.2962 | LR 0.0430


train-epoch-418:   0%|          | 0/8 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
scheduler.get_last_lr()

P